In [ ]:
%load_ext autoreload
%autoreload 2

GAN Training
====
An illustration of the GAN training data/loop

We'll take some patches of pure-noise (from the deep ocean, with low RMS - we expect these tiles to be pure noise, since the signal is slowly varying)
and train a GAN to reproduce them.

In [ ]:
"""
Take some tiles from our noisy data and display them, to illustrate what's going on
"""

import pathlib
import numpy as np
from current_denoising.plotting import maps
from current_denoising.generation import ioutils


# This is the "real" file the Laura made plots of in her paper doi:10.1017/eds.2023.41
filepath = pathlib.Path(
    "/home/mh19137/geog_rdsf/data/projects/SING/richard_stuff/Table2/currents/dtu18_eigen-6c4_do0280_rr0004_cs.dat"
)
assert filepath.exists()
data = ioutils.read_currents(filepath)

# Extract some tiles, rejecting the ones with high latitude or RMS
# An RMS of 0.20 is the 50th percentile in the dtu18_eigen-6c4_do0280_rr0004_cs data,
# and we use the same latitude threshold as Laura did
tile_size = 32  # in grid points
rms_threshold = 0.18
latitude_threshold = 64.0
rng = np.random.default_rng(1234)

tiles, indices = ioutils.extract_tiles(
    rng,
    data,
    num_tiles=512,
    max_rms=rms_threshold,
    max_latitude=64.0,
    tile_size=tile_size,
    return_indices=True,
)

In [ ]:
import matplotlib.pyplot as plt

tile_grid = np.ones_like(data) * np.nan

for tile, (y, x) in zip(tiles, indices):
    tile_grid[y : y + tile_size, x : x + tile_size] = tile

fig, axis = plt.subplots(figsize=(10, 5))
maps.imshow(tile_grid, axis=axis)
axis.imshow(
    1.4 * np.isnan(data),
    cmap=maps.clear2black_cmap(),
    extent=[-180, 180, -90, 90],
    vmin=0,
    vmax=1.4,
)
fig.colorbar(axis.images[0], ax=axis, label="m/s")

In [ ]:
"""
Turn our images into a dataloader with the right transforms
"""

from current_denoising.generation import dcgan

dataset = dcgan.TileLoader(tiles)

In [ ]:
"""
We might want to train lots of slightly different models, so write big function for the training + monitoring plots

"""

import pathlib

import torch
import matplotlib.cm as cm
import matplotlib.colors as colors
from torch.autograd import Variable
from scipy.stats import wasserstein_distance

from current_denoising.generation import dcgan
from current_denoising.plotting import training, img_validation


def _gp_plot(training_metrics, lambda_gp, plot_dir):
    fig = training_metrics.plot_gp_wd_ratio(lambda_gp)

    fig.savefig(plot_dir / "gp_wd.png")
    plt.close(fig)


def _grad_plot(train_metrics, g_lr, d_lr, plot_dir):
    fig = train_metrics.plot_param_gradients(g_lr, d_lr)
    fig.savefig(plot_dir / "grads.png")
    plt.close(fig)


def _grad_norm_plot(train_metrics, plot_dir):
    fig = train_metrics.plot_critic_grad_norms()
    fig.savefig(plot_dir / "grad_norm.png")
    plt.close(fig)


def _imgs_plot(imgs, plot_dir, title):
    fig = img_validation.show(imgs, cmap="turbo")
    mappable = cm.ScalarMappable(
        norm=colors.Normalize(vmin=0.0, vmax=1.4), cmap="turbo"
    )
    mappable.set_array([])
    fig.colorbar(mappable, ax=fig.axes)
    fig.suptitle(f"Generated images")
    fig.savefig(plot_dir / "generated.png")
    plt.close(fig)


def _hist_plot(imgs, dataloader, plot_dir, title):
    fig, axis = plt.subplots(1, 1, figsize=(8, 5))
    hist_kw = {
        "bins": np.linspace(-1, 1, 150),
        "density": True,
        "alpha": 0.5,
        "histtype": "step",
        "linewidth": 2,
    }
    img_validation.hist(imgs, axis=axis, **hist_kw, label="Generated images")
    img_validation.hist(
        next(iter(dataloader)),
        axis=axis,
        **hist_kw,
        label="Real images",
        linestyle="dashed",
    )
    axis.set_title(f"Image Hists - {title}")
    axis.legend()
    fig.tight_layout()
    fig.savefig(plot_dir / "hists.png")
    plt.close(fig)


def _fft_plot(imgs, plot_dir, title):
    # Plot FFTs of the generated and real images
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    img_validation.fft(imgs, axis=axes[0])
    axes[0].set_title("Generated images FFT")

    img_validation.fft(next(iter(dataloader)), axis=axes[1])
    axes[1].set_title(f"Real images FFT {title}")
    fig.tight_layout()
    fig.savefig(plot_dir / "ffts.png")
    plt.close(fig)


def train_gan(
    hyperparams: dcgan.GANHyperParams,
    dataloader: torch.utils.data.DataLoader,
    img_size: int,
    batch_size: int,
    output_dir: pathlib.Path,
) -> tuple[torch.nn.Module, torch.nn.Module]:
    """
    Train the GAN and make lots of debug plots

    Returns G + D and some metrics
    """
    metrics = {}

    # Train the GAN
    (generator, discriminator, train_metrics) = dcgan.train_new_gan(
        dataloader,
        hyperparams,
        "cuda",
        img_size=img_size,
        output_dir=output_dir,
    )

    title = f"g_lr={hyperparams.g_lr=}, d_lr={hyperparams.d_lr}"

    # Plot training losses
    fig = train_metrics.plot_scores()
    fig.suptitle(title)
    fig.tight_layout()
    fig.savefig(output_dir/ "losses.png")
    plt.close(fig)

    # Plot contributions of gradient penalty and Wasserstein distance to discriminator loss
    _gp_plot(train_metrics, hyperparams.lambda_gp, output_dir)

    # Plot gradients
    _grad_plot(train_metrics, hyperparams.g_lr, hyperparams.d_lr, output_dir)

    # Plot grad norm
    _grad_norm_plot(train_metrics, output_dir)

    # Generate some images and display them
    gen_imgs = dcgan._gen_imgs(generator, batch_size)
    _imgs_plot(gen_imgs, output_dir, title)
    _hist_plot(gen_imgs, dataloader, output_dir, title)
    _fft_plot(gen_imgs, output_dir, title)

    # Track metrics
    metrics["mean_wd_gp_ratio"] = (
        hyperparams.lambda_gp
        * np.mean(train_metrics.gradient_penalties, axis=1)
        / np.mean(train_metrics.wasserstein_dists, axis=1)
    ).mean()
    metrics["1-mean_gradient_ratio"] = abs(
        1
        - (
            (hyperparams.g_lr * train_metrics.generator_param_gradients)
            / (hyperparams.d_lr * train_metrics.critic_param_gradients)
        ).mean()
    )
    metrics["avg_grad_norm"] = np.mean(train_metrics.critic_interp_grad_norms, axis=1)[
        -50:
    ].mean()
    metrics["hist_wasserstein"] = wasserstein_distance(
        gen_imgs.detach().cpu().numpy().flatten(),
        next(iter(dataloader)).cpu().numpy().flatten(),
    )

    return generator, discriminator, metrics

In [ ]:
batch_size = 128

output_dir = pathlib.Path("outputs/gan_test/")
output_dir.mkdir(parents=True, exist_ok=True)

dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True
)
hyperparams = dcgan.GANHyperParams(
    n_epochs=21,
    g_lr=0.005,
    d_lr=0.0004,
    n_critic=5,
    lambda_gp=12,
    generator_latent_dim=64,
    n_discriminator_blocks=4,
)

train_gan(
    hyperparams,
    dataloader,
    img_size=tile_size,
    batch_size=batch_size,
    output_dir=output_dir,
)

Tuning
----
Here, we're looking for:
 - FID drops early (first 15 epochs)
 - Gradient norm near 1
 - $\lambda$ GP / |W| mostly between 0.1 and 0.6
 - Wasserstein distance peaks at first, then small positive plateau (without oscillations)

We will perform a random search over some of our parameters so that we can find a decent set to train with.

In [ ]:
"""
Train a lot of different GANs with different hyperparameters
"""

import pickle

plt.ioff()

n_runs = 50
n_epochs = 10

rng = np.random.default_rng(0)
metrics: list[dict] = []
out_dir = pathlib.Path("outputs/gan_tuning/")
out_dir.mkdir(parents=True, exist_ok=True)

for i in range(n_runs):
    hyperparams = dcgan.GANHyperParams(
        n_epochs=n_epochs,
        lr=10 ** rng.uniform(-6, -2),
        d_g_lr_ratio=rng.uniform(2, 32),
        n_critic=hyperparams.n_critic,
        lambda_gp=np.random.uniform(1, 20),
        n_fid_batches=4,
        generator_latent_dim=hyperparams.generator_latent_dim,
        n_discriminator_blocks=hyperparams.n_discriminator_blocks,
    )

    output_dir = out_dir / str(i)
    output_dir.mkdir(parents=True, exist_ok=True)

    _, _, tmp = train_gan(
        hyperparams,
        dataloader,
        img_size=tile_size,
        batch_size=batch_size,
        output_dir=output_dir,
    )

    tmp["lr"] = hyperparams.lr
    tmp["lambda_gp"] = hyperparams.lambda_gp
    tmp["d_g_lr_ratio"] = hyperparams.d_g_lr_ratio
    metrics.append(tmp)

with open("metrics.pkl", "wb") as f:
    pickle.dump(metrics, f)

In [ ]:
"""
Plot the metrics
"""

import pickle

# Lots of plots will have been made with interactive mode off, so close them all
# and then turn it back on
plt.close("all")
plt.ion()

with open("metrics.pkl", "rb") as f:
    metrics = pickle.load(f)

fig, axes = plt.subplots(2, 3, figsize=(8, 5))
param_names = ["lr", "lambda_gp", "d_g_lr_ratio"]
metric_names = [k for k in metrics[0] if k not in param_names]
for axis, key in zip(axes.flat, metric_names):
    axis.plot([d[key] for d in metrics], "o")
    axis.set_title(key)
    if "wd_gp" in key:
        axis.axhline(0.1, color="k", linestyle="dashed")
        axis.axhline(0.6, color="k", linestyle="dashed")
        axis.set_ylim(-3, 3)
    if "gradient_ratio" in key:
        axis.axhline(0.85, color="k", linestyle="dashed")
        axis.axhline(1.2, color="k", linestyle="dashed")
        axis.set_ylim(0, 2)
    if "grad_norm" in key:
        axis.axhline(0.9, color="k", linestyle="dashed")
        axis.axhline(1.1, color="k", linestyle="dashed")
        axis.set_ylim(0, 2)

fig.tight_layout()

In [ ]:
"""
Find the best runs according to each metric
"""

import pandas as pd
from IPython.display import display

metric_table = pd.DataFrame(metrics)

top_n = 5

print("FID: best, final")
display(metric_table.sort_values("best_fid").head(top_n)[param_names + ["best_fid"]])
display(metric_table.sort_values("final_fid").head(top_n)[param_names + ["final_fid"]])

# we want to find how close this one is to 0.3
print("Gradient penalty strength")
display(
    metric_table.assign(wd_gp_diff=lambda df: abs(df["mean_wd_gp_ratio"] - 0.3))
    .sort_values("wd_gp_diff")
    .head(top_n)[param_names + ["mean_wd_gp_ratio"]]
)

print("Gradient ratios")
display(
    metric_table.sort_values("1-mean_gradient_ratio").head(top_n)[
        param_names + ["1-mean_gradient_ratio"]
    ]
)

print("Gradient norm")
display(
    metric_table.assign(grad_norm_diff=lambda df: abs(df["avg_grad_norm"] - 1.0))
    .sort_values("grad_norm_diff")
    .head(top_n)[param_names + ["avg_grad_norm"]]
)

print("Wasserstein")
display(
    metric_table.sort_values("hist_wasserstein").head(top_n)[
        param_names + ["hist_wasserstein"]
    ]
)

In [ ]:
"""
Plot the metrics against the hyperparameters
"""

fig, axes = plt.subplots(len(metric_names), len(param_names), figsize=(12, 12))
for ax, key in zip(axes, metric_names):
    for axis, param in zip(ax, param_names):
        sc = axis.scatter(
            [d[param] for d in metrics],
            [d[key] for d in metrics],
            alpha=0.7,
            label=param,
            s=1,
        )


for axis, title in zip(axes[0], param_names):
    axis.set_title(title)

for axis, title in zip(axes[:, 0], metric_names):
    axis.set_ylabel(title)
    axis.set_xscale("log")

for axis in axes[2]:
    axis.set_ylim(0, 1)
    axis.axhline(0.6, color="k", linestyle="dashed")
    axis.axhline(0.1, color="k", linestyle="dashed")

for axis in axes[4]:
    axis.axhline(0.8, color="k", linestyle="dashed")
    axis.axhline(1.2, color="k", linestyle="dashed")


fig.tight_layout()

In [ ]:
"""
Now that we have a sensible d_g_lr_ratio, train a longer run
"""

assert False

In [ ]:
import torch

batch_size = 128
config = {
    "n_epochs": 300,
    "n_critic": 5,
    "lambda_gp": 13.7,
    "learning_rate": 0.005,
    "d_g_lr_ratio": 12.7,
    "latent_dim": 64,
    "img_size": tile_size,
    "channels": 1,
    "batch_size": batch_size,
    "dataloader": torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=True, num_workers=8
    ),
    "output_dir": pathlib.Path("outputs/gan_final/"),
    "plot_interval": 10,
}

generator, discriminator, _ = train_gan(config, config["output_dir"])

In [ ]:
torch.save(generator.state_dict(), config["output_dir"] / "generator_final.pth")
torch.save(discriminator.state_dict(), config["output_dir"] / "discriminator_final.pth")